# Doc2Vec

O doc2vec, ou Paragraph Embedding foi proposto em [[1](https://arxiv.org/pdf/1405.4053v2.pdf)]. Osautores são os mesmos do word2vec, e a semelhança entre as técnicas também. A extração do doc2vec não se liimita a documentos, podendo ser usada para qualquer sequência de palavras: títulos, parágrafos, tweets, entre outros. A ideia aqui é justamente representar documentos de forma a preservar características como a ordem das palvras, a qual se perde ao usar o BOW ou a média dos w2v.

Para isso, as técnicas usadas para d2v são bastante semelhantes ao w2v (SkipGram e CBOW). A diferença é que existe um vetor retido durante todo o processo de treinamento, o **Paragraph Vector**. Esse vetor é usado em todas as etapas de treinamento, ou seja, para toda janela esse vetor está lá, funcionando como um vetor de memória. Ao final do treinamento, temos um vetor que pode representar toda a sequência de palavras.

<div text-align="center">
    <img src="imgs/cbow.png">
    <img src="imgs/skip.png">
</div>

Portanto, neste notebook vamos analisar como funcionaria nosso modelo usando essas carcterísticas. Portanto, vamos verificar qual a diferença entre modelar o documento inteiro ou cada palavra dele.

In [1]:
import numpy as np
import pandas as pd
import gensim
from context import fakenews
from fakenews import preprocess as pre
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier

Primeiro, vamos carregar e fazero preprocessamento dos dados, assim como fizemos na aula03. Na verdade, tudo que vem a seguir é praticamente o mesmo código da aula03, com poucas diferenças.

In [2]:
base = '/home/thales/dev/fakenews/'
news, labels = pre.run(base + 'data/Fake.csv', base + 'data/True.csv')
pre.truncate_news(news)

Succesfully read data from:
Fakes: /home/thales/dev/fakenews/data/Fake.csv
Reals: /home/thales/dev/fakenews/data/True.csv
Removing rows without text...
Removing publisher information...
Adding class column...
Merging fakes and reals
Merging titles and bodies...
Removing subjects and date...
Tokenizing data...
Truncating at 869


Agora, vamos gerar o modelo doc2vec. A implementação do gensim usa o DBOW (imagem da esquerda no início do notebook). Podemos escolher o tamanho da janela, quantidade de *workers* (jobs), e a dimensão do vetor. Vamos usar uma janela de 5, e dimensão 100. Podem alterar os parâmetros e ver as diferenças nos resultados caso achem necessário. 

In [3]:
from gensim.models.doc2vec import TaggedDocument, Doc2Vec

docs = [TaggedDocument(new, [doc_ID]) for doc_ID, new in enumerate(news)]
d2v = Doc2Vec(docs, vector_size=100, window=5, workers=4, min_count=1)

Com o modelo treinado, podemos liberar um cache gerado pelo gensim. Isso é opcional, no nosso caso vamos ignorar isso e vamos direto para a inferência. A inferência é feita usando o método `.infer_vector()` ou `.dv['docID']`. 
Abaixo podemos ver um exemplo de ambos.

In [4]:
d2v.infer_vector(news[0])

array([-0.27324262,  0.3813901 , -0.38319057,  0.11611375,  0.37417048,
       -0.18729281,  0.05496622, -0.4384446 ,  0.51537293, -0.52938557,
       -0.39907634,  0.43492252,  0.445951  , -0.22944959, -0.23032373,
       -0.2397254 ,  0.05542371, -0.38546386, -0.31398943, -0.7509162 ,
        0.08827436, -0.3443745 , -0.03105517,  0.13167888,  0.226244  ,
        0.10663103, -0.00856167,  0.04230334, -0.18538818,  0.3780418 ,
        0.60150486,  0.0300507 ,  0.7336161 ,  0.00119405,  0.46444464,
       -0.0802251 ,  0.3814574 , -0.40198985,  0.79524857,  0.61929154,
       -0.8601831 , -0.23806645,  0.3258657 , -0.17329612, -0.26873192,
        0.13025488, -0.51481885,  0.7612322 , -0.04145912, -0.1221593 ,
       -0.77204657,  0.18659645,  0.78452194,  0.3940642 , -0.02370133,
       -0.2669998 ,  0.24086395, -0.57401484,  0.16611397, -0.32364413,
        0.18323654, -0.5698154 ,  0.13754262,  0.5269307 ,  0.29959667,
       -0.07940871, -0.16247198,  0.4768695 ,  0.7341448 ,  0.06

In [5]:
d2v.docvecs[0]

array([-0.24089529,  0.0559379 , -0.21807459,  0.19554463,  0.3699122 ,
       -0.00879263,  0.00376805, -0.4713308 ,  0.42585188, -0.46749908,
       -0.2383313 ,  0.29708573,  0.3939724 , -0.17290026, -0.2450358 ,
       -0.2584335 ,  0.07154153, -0.14825585,  0.00512494, -0.25686622,
        0.23380998, -0.01894146,  0.0992048 , -0.03271559,  0.26636344,
       -0.09601244,  0.0522934 , -0.1950375 ,  0.08511513,  0.24587701,
        0.43570092,  0.16508323,  0.7687626 ,  0.06068579,  0.44363174,
       -0.20286936,  0.06348096, -0.683628  ,  0.7652566 ,  0.46952638,
       -0.7472243 , -0.21879041,  0.18656905, -0.06339938, -0.18011089,
        0.05998008, -0.55308354,  0.6702631 , -0.01333776,  0.01400774,
       -0.6741057 , -0.05296347,  0.44489664,  0.4068878 ,  0.1300171 ,
       -0.2478381 ,  0.2928713 , -0.31014508,  0.20275319, -0.18562911,
        0.36592728, -0.51171285,  0.05356467,  0.24336219,  0.08907641,
       -0.0480834 , -0.28805616,  0.28691784,  0.64603454, -0.11

Como podemos ver, o vetor gerado durante o treinamento é obtido a partir do ID do documento, enquanto que a inferência gera um novo vetor baseado nos dados de treinamento.

In [6]:
d2v.save('fakes-d2v')

Da mesma forma que fizemos para o w2v, podemos persistir o modelo no disco para uso futuro. A seguir, transformamos os dados (palavras/documentos) nos seus respectivos vetores. 

In [7]:
doc_ids = np.arange(44267, dtype=np.int32)
docvecs = np.array([d2v.docvecs[id_] for id_ in doc_ids])
docvecs.shape

(44267, 100)

Com as features representando cada documento, agora vamos dividir nossa base em treino/test. Vamos repetir as mesmas configurações usadas na extração do word2vec. Dessa forma, podemos comparar os resultados diretamente com os experimentos anteriores.

In [8]:
splits = train_test_split(docvecs, labels, test_size=0.3, shuffle=True)
news_trn, news_test, lbl_trn, lbl_test = splits
news_trn[:5, (0, 1, 97, 98, 99)]

array([[-0.19650473,  0.1908642 ,  0.19341165,  0.32731286,  0.25321385],
       [-0.22313836,  0.23449564, -0.11937473,  0.15155253, -0.12210088],
       [ 0.12032755,  0.3766597 ,  0.3611389 ,  0.05942689, -0.07596436],
       [ 0.0350279 , -0.26440144,  0.12007716,  0.40381876, -0.43361357],
       [-0.00980167, -0.0112812 ,  0.02000316,  0.08399785, -0.02749721]],
      dtype=float32)

Agora, escolhemos quais parâmetros vamos fazer os experimentos usando o KNN. 

In [9]:
models = [KNeighborsClassifier()]
pgrids = [
    {'n_neighbors': [4, 8, 16], 'p': [2, 3]}
]

Finalmente, fazemos o fine-tuning com os dados de treino com um GridSearch usando uma validação cruzada de 3 folds.

In [ ]:
best_models = []
for model, grid in zip(models, pgrids):
    optimizer = GridSearchCV(model, grid, cv=3)
    optimizer.fit(news_trn, lbl_trn)
    best_models.append(optimizer.best_estimator_)

best_models

Tendo escolhido o melhor modelo, agora só precisamos testar!

In [ ]:
for model in best_models:
    testacc = model.score(tst_vecs, labels_tst)
    print(testacc)

Idealmente, outras rodadas de experimentos seriam executadas. Dessa vez, fazendo alterações nas features. Mudando a dimensão, o tamanho da janela e outros parâmetros. Assim, podemos identificar casos onde uma featurepode ser melhor que a outra.